In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import interpolate
from scipy import spatial

from utils import remove_hidden_points, edblquad
from plotting import set_axes_equal, set_defense_context

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
def sphere(r, samples=1000):
    # https://stackoverflow.com/a/44164075/15005103
    i = np.arange(0, samples, dtype=float) + 0.5
    phi = np.arccos(1 - 2 * i / samples)
    theta = np.pi * (1 + np.sqrt(5)) * i
    x = r * np.cos(theta) * np.sin(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(phi)
    return np.c_[x, y, z]

In [ ]:
# generate coordinates of a sphere

r = 5  # cm
samples = 10000
xyz = sphere(r, samples)

In [ ]:
# create 2 synthetic absorbed power density patterns

mask = np.where(xyz[:, 0] > 0)[0]  # the wave propagates in the x-direction
y = xyz[mask, 1]
z = xyz[mask, 2]

center_1 = [0, 0]
radius_1 = 0.5
region_1 = np.sqrt(((y - center_1[0]) / 1.5) ** 2 + (z - center_1[1]) ** 2)
apd_1 = 15.1 * np.exp(-(region_1 / radius_1) ** 2)

center_2 = [2.5, -1]
radius_2 = 1.5
region_2 = np.sqrt((y - center_2[0]) ** 2 + (z - center_2[1]) ** 2)
apd_2 = 10 * np.exp(-(region_2 / radius_2) ** 2)

apd = np.zeros((xyz.shape[0], ))
apd[mask] = apd_1 + apd_2

# Incident wave direction is known

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz.T, s=1, c=apd)
    fig.colorbar(s, ax=ax, pad=0.075, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.quiver(9, 0, 0, -2.5, 0, 0, lw=1, color='w')
    ax.text(9, 0, 0.5, s='$\\vec k$', color='w', zdir='x')
    ax.set(xlabel='$x$ [cm]', ylabel='$y$ [cm]', zlabel='$z$ [cm]')
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(10, 30)
    fig.tight_layout()
    plt.show()

In [ ]:
# extract visible points from the assumed direction of incidence

center = np.mean(xyz, axis=0)
diameter = np.linalg.norm(xyz.max(axis=0) - xyz.min(axis=0))
pov = center.copy()
pov[0] += diameter
idx_visible = remove_hidden_points(xyz, pov, np.pi)
xyz_visible = xyz[idx_visible]
apd_visible = apd[idx_visible]

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz_visible.T, c=apd_visible, s=1)
    fig.colorbar(s, ax=ax, pad=0, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
           xticks=[], xticklabels=[],
           yticks=[-r, 0, r],
           zticks=[-r, 0, r])
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(0, 0)
    fig.tight_layout()
    plt.show()

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*xyz_visible.T, c=apd_visible, s=1)
    fig.colorbar(s, ax=ax, pad=0, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.quiver(9, 0, 0, -2.5, 0, 0, lw=1, color='k')
    ax.text(8, 0, 0.5, s='$\\vec k$', color='k', zdir='x')
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.set_axis_off()
    ax.view_init(0, 90)
    fig.tight_layout()
    plt.show()

In [ ]:
# extract 2x2 cm2 areas in a plane perpendicular to k vector (x-direction)

area = 4  # cm2
a = np.sqrt(area)
res = {'p': [], 'nbhd': [], 'area_conf': [], 'apd': [], 'sapd': []}
for p in xyz_visible:
    _bbox = [p[1]-a/2, p[1]+a/2, p[2]-a/2, p[2]+a/2]
    _idx_bbox = np.where((xyz_visible[:, 1] >= _bbox[0])
                         & (xyz_visible[:, 1] <= _bbox[1])
                         & (xyz_visible[:, 2] >= _bbox[2])
                         & (xyz_visible[:, 2] <= _bbox[3]))[0]
    _nbhd = xyz_visible[_idx_bbox]
    _apd = apd_visible[_idx_bbox]
    _hull = spatial.ConvexHull(_nbhd[:, 1:])
    _hull_area = _hull.volume  # for 2-D `volume` results with an area
    rpd = np.abs(_hull_area - area) / area
    if rpd > 0.1:
        continue  # skip if the difference is too large
    else:
        fi = interpolate.SmoothBivariateSpline(*_nbhd[:, 1:].T,  # yz
                                               _nbhd[:, 0],      # x
                                               kx=5, ky=5)
        _n = np.c_[fi(*_nbhd[:, 1:].T, dx=1, grid=False),   # dy
                   fi(*_nbhd[:, 1:].T, dy=1, grid=False),   # dz
                   np.ones((_nbhd.shape[0]))]
        _n_mag = np.linalg.norm(_n, axis=1)
        _area_conf = edblquad(_nbhd[:, 1:], _n_mag)  # conformal area
        _sapd = edblquad(_nbhd[:, 1:], _apd) / _area_conf  # surface integral
        res['p'].append(p)
        res['nbhd'].append(_nbhd)
        res['area_conf'].append(_area_conf)
        res['apd'].append(_apd)
        res['sapd'].append(_sapd)
res_df = pd.DataFrame(res)

In [ ]:
# spatial maximum

_, ctrl_nbhd, ctrl_area_conf, ctrl_apd, ctrl_sapd = res_df[
    res_df['sapd'] == res_df['sapd'].max()
].to_numpy()[0].T

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*ctrl_nbhd.T, c=ctrl_apd)
    fig.colorbar(s, ax=ax, pad=0, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
           title=(r'$s\text{PD}_{max, \; 4 \; \text{cm}^2}$ = '
                  f'{ctrl_sapd:.2f} W/m$^2$'),
           xticks=[], xticklabels=[])
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(0, 0)
    plt.show()

In [ ]:
# central point

ctrl_idx = np.where(np.isclose(res['p'], np.array([5, 0, 0])))[0]
_, ctrl_nbhd, ctrl_area_conf, ctrl_apd, ctrl_sapd = res_df.loc[
    ctrl_idx].to_numpy()[0].T

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(5, 5))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*ctrl_nbhd.T, c=ctrl_pd)
    fig.colorbar(s, ax=ax, pad=0, shrink=0.5,
                 label='power density [W/m$^2$]')
    ax.set(xlabel='', ylabel='$y$ [cm]', zlabel='$z$ [cm]',
           title=(r'$s\text{PD}_{max, \; 4 \; \text{cm}^2}$ = '
                  f'{ctrl_sapd:.2f} W/m$^2$'),
           xticks=[], xticklabels=[])
    ax.xaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.yaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.zaxis.set_pane_color((1.0, 1.0, 1.0, 1.0))
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    ax.view_init(0, 0)
    plt.show()

# Incident wave direction is unknown - wip

In [ ]:
area = 4  # square-shaped surface in cm2
r = np.sqrt(A / np.pi)

query_point = xyz[np.where(color == color.max())[0][0]]
bbox = [query_point[1]-a/2, query_point[1]+a/2,
        query_point[2]-a/2, query_point[2]+a/2]

tree = spatial.KDTree(xyz)
nbh_idx = tree.query_ball_point([query_point], r)[0]
query_nbh = xyz[nbh_idx]

In [ ]:
with set_defense_context():
    fig = plt.figure(figsize=(6, 6))
    ax = plt.axes(projection ='3d')
    s = ax.scatter(*query_nbh.T, c=color[nbh_idx])
    ax.set(xlabel='x', ylabel='y', zlabel='z')
    ax.view_init(0, 0)
    ax.set_box_aspect([1, 1, 1])
    ax = set_axes_equal(ax)
    fig.colorbar(s, ax=ax, pad=0.1, shrink=0.5, label='PD')
    plt.show()